# Production ready crosslight detection

### Lets create our model and load the weights

In [ ]:
import display

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import img_to_array, load_img # Do I really need this?

from picamera import PiCamera, array
from time import sleep
from numpy import around

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 512, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.load_weights('ciffer.h5')

### Raspberry camera logic

In [ ]:
with PiCamera() as camera:
    camera.resolution = (128, 512)
    sleep(1)
    number_found = False
    
    
    display.one()
    with array.PiRGBArray(camera) as output:
        while number_found == False:
            camera.capture(output, 'rgb')
            img = output.array.astype(float)
            img = img.reshape((1,) + img.shape)
            prediction = around(model.predict(img, 3))
            print(prediction)
            
            # Found logic
            
            output.truncate(0)